In [1]:
from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

pd.set_option("display.max_rows", 300)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

## noise analysis

In [2]:
trn_df = pd.read_csv('../inputs/nes_info/e080_dataset_trn_df.csv').dropna()
display(trn_df.shape, trn_df.head())

(27423, 12)

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going",7,9
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad,sooo sad,2,3
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me,bullying me,2,2
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone,leave me alone,3,3
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,","sons of ****,",3,4


In [10]:
magic_trn_df = trn_df[trn_df.text.str.contains('    ')]
magic_trn_df.head()

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
18,af3fed7fc3,is back home now gonna miss every one,onna,negative,is back home now gonna miss every one,onna,False,miss,gonna,onna,0,0
88,bffa3ddd61,i miss you bby wish you were going tomorr...,i miss you bby,negative,i miss you bby wish you were going tomorrow to...,i miss you bby,False,i miss,i miss you bby,i miss you bby,4,5
110,9c4817f73b,if u have a friendster add me!!!!!!!!! ...,if u have a friendster add me!!!!!!!!! ...,neutral,if u have a friendster add me!!!!!!!!! my emai...,if u have a friendster add me!!!!!!!!! my emai...,False,if u have a friendster add me!!!!!!!!! my ema...,if u have a friendster add me!!!!!!!!! my ema...,if u have a friendster add me!!!!!!!!! my ema...,12,22
226,151c10cc39,JONAS BROTHERS - Live to party. ...,JONAS BROTHERS - Live to party. ...,positive,JONAS BROTHERS - Live to party. It`s rocking s...,JONAS BROTHERS - Live to party. It`s rocking s...,False,"i love the song,",jonas brothers - live to party. it`s rocking ...,jonas brothers - live to party. it`s rocking ...,10,15
430,5010f92b03,Took a shift tomorrow. I don`t really feel ...,I don`t really feel like,negative,Took a shift tomorrow. I don`t really feel lik...,I don`t really feel like,False,i don`t really feel like working right now.,i don`t really feel like,i don`t really feel like,5,7


In [11]:
def get_char_targets(src_text, selected_text):
    src_text = row.text
    selected_text = row.selected_text
    idx0, idx1 = None, None
    for i, char in enumerate(src_text):
        if char == selected_text[0] and src_text[i: i+len(selected_text)] == selected_text:
            idx0, idx1 = i, i + len(selected_text)
            break
    assert src_text[idx0:idx1] == selected_text

    char_targets = [0] * len(src_text)
    if idx0 is not None and idx1 is not None:
        for i in range(idx0, idx1):
            char_targets[i] = 1

    return char_targets

def preprocess(text):
    maxlen = len(text)
    text = ' '.join(text.strip().split())
    text += ' ' * (maxlen - len(text))
    return text

In [14]:
for i, row in magic_trn_df.sample(20, random_state=72).iterrows():
    print(row['selected_text'])
    with pd.option_context('display.max_columns', None):
        df = pd.DataFrame(dict(
            char_targets=get_char_targets(row.text, row.selected_text),
            raw_text=list(row.text),
            fixed_text=list(preprocess(row.text)),
        )).T
        display(df)

Thanks


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
char_targets,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
raw_text,T,h,a,n,k,s,,,,,_,T,w,i,t,0,1
fixed_text,T,h,a,n,k,s,,_,T,w,i,t,0,1,,,


I`m sa


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92
char_targets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0
raw_text,H,a,d,,a,,c,r,a,z,y,,n,i,g,h,t,",",,l,o,s,t,,k,e,y,s,",",,w,a,l,k,e,d,,h,o,m,e,",",,m,i,s,s,e,d,,m,y,,p,a,p,i,`,s,,c,a,l,l,s,,f,r,o,m,,t,h,e,,s,w,i,z,z,y,.,,,,,I,`,m,,s,a,d
fixed_text,H,a,d,,a,,c,r,a,z,y,,n,i,g,h,t,",",,l,o,s,t,,k,e,y,s,",",,w,a,l,k,e,d,,h,o,m,e,",",,m,i,s,s,e,d,,m,y,,p,a,p,i,`,s,,c,a,l,l,s,,f,r,o,m,,t,h,e,,s,w,i,z,z,y,.,,I,`,m,,s,a,d,,,


was gonna DM you but it says you`re not following me!    lol  after all I`ve done!  hah


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
char_targets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
raw_text,i,,b,a,s,i,c,a,l,l,y,,w,a,n,t,,t,h,e,,w,o,r,l,d,,t,o,,k,n,o,,t,h,a,t,,i,,h,a,v,e,,t,h,e,,B,E,S,T,,b,e,s,t,,f,r,i,e,n,d,,,,,l,o,v,e,,h,e,r,,<,3
fixed_text,i,,b,a,s,i,c,a,l,l,y,,w,a,n,t,,t,h,e,,w,o,r,l,d,,t,o,,k,n,o,,t,h,a,t,,i,,h,a,v,e,,t,h,e,,B,E,S,T,,b,e,s,t,,f,r,i,e,n,d,,l,o,v,e,,h,e,r,,<,3,,,


hope all is ok


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
char_targets,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
raw_text,,,,,h,o,p,e,,a,l,l,,i,s,,o,k,.
fixed_text,h,o,p,e,,a,l,l,,i,s,,o,k,.,,,,


a riot


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
char_targets,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
raw_text,,,Y,o,u,r,,s,u,c,h,,a,,r,i,o,t,!,!,,,,,Y,o,u,,g,o,,g,i,r,l,!,!
fixed_text,Y,o,u,r,,s,u,c,h,,a,,r,i,o,t,!,!,,Y,o,u,,g,o,,g,i,r,l,!,!,,,,,


In [15]:
import re

In [72]:
re.match(r' [^ ]', ' b   ')

<re.Match object; span=(0, 2), match=' b'>

In [49]:
re.match(r'[apple]', ' a  apple')

In [69]:
for i, row in trn_df.iterrows():
    a = re.match(r' [^ ]', row['text'])
    if a is not None:
        print(f':{row["text"]}:')
        print(a)
        break

: I`d have responded, if I were going:
<re.Match object; span=(0, 2), match=' I'>


In [75]:
re.match(r' apple', ' apple  ')

<re.Match object; span=(0, 6), match=' apple'>

## 

In [76]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

In [78]:
offsets = tokenizer.encode('apple pie').offsets
offsets

[(0, 5), (5, 9)]

In [80]:
'apple pie'[offsets[0][0]:offsets[0][1]]

'apple'